## Setup


In [ ]:
import requests
import json
import base64
import brikasutils as bu
import test_utils as tu
import random
from aiohttp import ClientTimeout  # noqa: F401
from dotenv import load_dotenv
from typing import Dict, List
import os
from pathlib import Path
from aiohttp import FormData

from faker import Faker
fake = Faker()
Faker.seed("MEALMEAP")

import importlib
importlib.reload(bu)
importlib.reload(tu)

from brikasutils import RequestTemplate as RT

load_dotenv(override=True)
ADMIN_API_KEY = os.getenv('ADMIN_API_KEY') or 'undefined'

HEADERS_CONTENT_JSON = {'Content-Type': 'application/json'}

# Endpoint Base URLs - DEVELOPMENT - FastAPI
# BASE_URL_AUTH = 'http://91.99.7.176:8000/auth'
# BASE_URL_FUNCTIONS = 'http://91.99.7.176:8000'
BASE_URL_AUTH = 'http://localhost:8000/auth'
BASE_URL_FUNCTIONS = 'http://localhost:8000'

## Users


### Setup Users


In [ ]:
MIME_TYPE = 'image/jpeg'

IMAGE_SQ_PERSON_1 = 'img/test-img-sq-person-1.jpg'
IMAGE_SQ_PERSON_2 = 'img/test-img-sq-person-2.jpg'
IMAGE_SQ_PERSON_3 = 'img/test-img-sq-person-3.jpg'
IMAGE_SQ_PERSON_4 = 'img/test-img-sq-person-4.jpg'
IMAGE_NONSQ_PERSON_1 = 'img/test-img-nonsq-person-1.jpg'
IMAGE_NONSQ_PERSON_2 = 'img/test-img-nonsq-person-2.jpg'


IMAGE_SQ_GROUP_DORM = 'img/test-img-sq-group-dorm.jpg'
IMAGE_SQ_GROUP_DTU = 'img/test-img-sq-group-dtu.jpg'
IMAGE_SQ_GROUP_LOGO = 'img/test-img-sq-group-logo.jpg'


users = [
    {
        "email": "airidas.brikas@gmail.com",
        "password": "1234567",
        "first_name": "Airidas",
        "last_name": "Brikas",
        "image": IMAGE_SQ_PERSON_1,
        # "token": str
        # "id": str
        # "groups": ["..."]

    },
    {
        "email": "rares@t.com",
        "password": "1234567",
        "first_name": "Rares",
        "last_name": "Diaconescu",
        "image": IMAGE_SQ_PERSON_2,
    },
    {
        "email": "anna@t.com",
        "password": "1234567",
        "first_name": "Anna",
        "last_name": "Kowalska",
        "image": IMAGE_SQ_PERSON_3,
    },
    {
        "email": "elias@t.com",
        "password": "1234567",
        "first_name": "Elias",
        "last_name": "Lunoe",
        "image": IMAGE_SQ_PERSON_4
        # No image 
    },
    {
        "email": "david@t.com",
        "password": "1234567",
        "first_name": "David",
        "last_name": "Jaic",
        "image": IMAGE_SQ_PERSON_3,
    },
    {
        "email": "george@t.com",
        "password": "1234567",
        "first_name": "George",
        "last_name": "Andrei",
        "image": IMAGE_SQ_PERSON_4,
    },


]

# Apply defaults
def apply_user_defaults():
    for user in users:
        if "send_welcome_email" not in user:
            user["send_welcome_email"] = False
        if "image" not in user:
            user["image"] = None
apply_user_defaults()

### Add additional, special users


In [ ]:
# Add new users
add_users = [
    # EXTERNALLY INVITED LATER 
    {
        "email": "ninjatxgaming@gmail.com",
        "password": "1234567",
        "first_name": "Airidas",
        "last_name": "Brikas2",
        # "image": image,
    },
]

for user in add_users:
    if not any(u['email'] == user['email'] for u in users):
        users.append(user)
    else:
        for u in users:
            if u['email'] == user['email']:
                # Delete the user from the list
                users.remove(u)
                users.append(user)
                print(f"User {user['email']} replaced")
                break


apply_user_defaults()


##### _Add additional generated users_


In [ ]:
# Generate X amount of groups using faker
NUM_USERS = 150  # Set the number of groups you want to generate
BATCH_ID = "gen-users-1"

add_users = []
for i in range(NUM_USERS):
    add_users.append({
        "email": fake.email(),
        "password": "1234567",
        "first_name": fake.first_name(),
        "last_name": fake.last_name(),
        "batch_id": BATCH_ID,
    })

# Add generated groups to the main groups list and apply defaults
users.extend(add_users)
apply_user_defaults()
print(f"Added {NUM_USERS} generated users.")

### Register New users


In [ ]:
ONLY_NON_EXISTING = True

SEND_EMAIL = False
rts = []
for user in users:
    if ONLY_NON_EXISTING and 'id' in user and user["id"] != None:
        continue

    def callback(response: bu.CustomResponse, user=user):        
        if response.status == 200:
            print(f"{response.status}: {user['email']}")
            r = response.json()
            if r != None:
                user['token'] = r["access_token"]
                user['id'] = r["user_id"]
        else:
            print(f"{response.status}: {user['email']}, response: {response.text}")

    rts.append(
        RT(
            method="post",
            url=f'{BASE_URL_AUTH}/register',
            json={
                "email": user['email'],
                "password": user['password'],
                "send_email": True if user['send_welcome_email'] else SEND_EMAIL,
                "test_id": user['batch_id'] if 'batch_id' in user else None,
            },
            timeout=60,
            headers=HEADERS_CONTENT_JSON,
            callback=callback,
        )
    )

responses = bu.run_async_requests(rts, verbose=False, max_concurrent=5)

### Log in Users


In [ ]:
ONLY_NON_LOGGED_IN = False
AUTH_ENDPOINT = f'{BASE_URL_AUTH}/token'

rts = []

for user in users:
    if ONLY_NON_LOGGED_IN and 'token' in user and user['token'] != None:
        continue

    if "email" in user and "password" in user:
        def callback(response: bu.CustomResponse, user=user):
            if response.status == 200:
                print(f"{response.status}: {user['email']}")
                r = response.json()
                if r != None:
                    user['token'] = r["access_token"]
                    user['id'] = r["user_id"]
            else:
                print(f"{response.status}: {user['email']}, response: {response.text}")

        rts.append(
            RT(
                method="post",
                url=AUTH_ENDPOINT,
                json={
                    "email": user['email'],
                    "password": user['password'],
                },
                timeout=30,
                headers=HEADERS_CONTENT_JSON,
                callback=callback
            )
        )

responses = bu.run_async_requests(rts, verbose=False)



### Test Login


In [ ]:
# Test /me endpoint
rts = []
for user in users:
    if "token" not in user or user['token'] == None:
        print(f"User {user['email']} does not have token, skipping /me test")
        continue

    def callback(response: bu.CustomResponse, user=user):
        if response.status == 200:
            r = response.json()
            if r == None:
                print(f"{response.status}: /me {user['email']}, response: None")
                return
            print(f"{response.status}: /me {user['email']} -> id: {r['id']}")
        else:
            print(f"{response.status}: /me {user['email']}, response: {response.text}")

    rts.append(
        RT(
            method="get",
            url=f'{BASE_URL_AUTH}/users/me',
            timeout=10,
            headers={
                'Authorization': f'Bearer {user["token"]}'
            },
            callback=callback
        )
    )
responses = bu.run_async_requests(rts, verbose=False)

### Update User


In [ ]:
only_users = [
    # tu.get_user_from_email("petyo@t.com", users)['id'],
]

rts = []
for user in users:

    if len(only_users) > 0 and user['id'] not in only_users:
        continue

    def callback(response: bu.CustomResponse, user=user):
        print(f"{response.status}: {tu.emailOrPhone(user)}, response: {response.text}")


    rts.append(
        RT(
            method="post",
            url=f'{BASE_URL_FUNCTIONS}/users/{user["id"]}',
            json={
                "first_name": user['first_name'],
                "last_name": user['last_name'],
                # "image": tu.encode_base64_image_or_none(user['image']),
            },
            timeout=30,
            headers= tu.auth_json_headers(user['token']),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False, max_concurrent=10)

### Update Profile Image


In [ ]:
user_image_updates = [
    {
        "action_caller": tu.get_user_from_email("airidas.brikas@gmail.com", users),
        "image": "img/test-img-sq-person-1.jpg",
    }
]

# Add all users with images to the list
for user in users:
    if 'image' in user and user['image'] is not None:
        user_image_updates.append({
            "action_caller": user,
            "image": user['image'],
        })

rts = []

for update in user_image_updates:
    user = update['action_caller']

    form = FormData()
    form.add_field(
        'image',
        open(update['image'], 'rb'),
        filename=Path(update['image']).name,
        content_type="image/jpeg" 
    )

    def callback(response: bu.CustomResponse, user=user):
        if response.status == 200:
            r = response.json()
            if r != None:
                print(f"{response.status}: {tu.emailOrPhone(user)}", 
                      f"image_url: {r['image_url']}")
            else:
                print(f"{response.status}: {tu.emailOrPhone(user)}, no response data")
        else:
            print(f"{response.status}: {tu.emailOrPhone(user)}, response: {response.text}")
    
    rts.append(
        RT(
            method="post",
            url=f'{BASE_URL_FUNCTIONS}/users/{user["id"]}/profile-image',
            data=form,
            timeout=30,
            headers={
                "Authorization": f"Bearer {user['token']}",
            },
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False, max_concurrent=3)

### Change Password


In [ ]:
queries = [
    {
        "action_caller": tu.get_user_from_email("airidas.brikas@gmail.com", users),
        "current_password": "1234567890",
        "new_password": "1234567",
    },
    {
        "action_caller": tu.get_user_from_email("rares@t.com", users),
        "current_password": "1234567",
        "new_password": "12345678",
    }
]

rts = []
for query in queries:
    user = query['action_caller']

    def callback(response: bu.CustomResponse, user=user, query=query):
        if response.status == 200:
            r = response.json()
            print(f"{response.status}: {tu.emailOrPhone(user)}. Updating local records.")
            user['password'] = query['new_password']
        else:
            print(f"{response.status}: {tu.emailOrPhone(user)}, response: {response.text}")
    
    rts.append(
        RT(
            method="post",
            url=f'{BASE_URL_FUNCTIONS}/users/me/change-password',
            json={
                "current_password": query['current_password'],
                "new_password": query['new_password'],
            },
            timeout=30,
            headers= tu.auth_json_headers(user['token']),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

### Search Users


In [ ]:
queries = [
    {
        "action_caller": { "email": "david@t.com"},
        "query": "rar",
    }
]

rts = []
for query in queries:
    user = tu.get_user_from_email(query['action_caller']['email'], users)

    def callback(response: bu.CustomResponse, user=user):
        if response.status_code == 200:
            r = response.json()
            if r != None:
                print(f"{response.status}: {user['email']} -> {len(r['results'])} results")
                for res in r['results']:
                    print(f" - {res['first_name']} {res['last_name']} ({res['email']})")
        else:
            print(f"{response.status}: {user['email']}, response: {response.text}")

    rts.append(
        RT(
            method="get",
            url=f'{BASE_URL_FUNCTIONS}/users',
            params={
                "q": query["query"],
            },
            timeout=10,
            headers=tu.auth_json_headers(user["token"]),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

### Get User


In [ ]:
queries = [
    {
        "action_caller_id": tu.get_user_from_email("rares@t.com", users)['id'],
        "user_id": tu.get_user_from_email("airidas.brikas@gmail.com", users)["id"],
    },
]

rts = []
for query in queries:
    user = tu.get_user_from_id(query['action_caller_id'], users)

    def callback(response: bu.CustomResponse, user=user, query=query):
        if response.status_code == 200:
            r = response.json()
            if r != None:
                print(f"{response.status}: {user['email']} -> {query["user_id"]},")
                print(json.dumps(r, indent=2))
        else:
            print(f"{response.status}: {user['email']} -> {query["user_id"]}, response: {response.text}")

    rts.append(
        RT(
            method="get",
            url=f'{BASE_URL_FUNCTIONS}/users/{query["user_id"]}',
            timeout=10,
            headers=tu.auth_json_headers(user["token"]),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

### Delete User


In [ ]:
# Delete user account
queries = [
    {
        "action_caller": tu.get_user_from_email("patryk@t.com", users),
    }
]

rts = []
for query in queries:
    user = query['action_caller']

    def callback(response: bu.CustomResponse, user=user):
        if response.status_code == 200:
            print(f"{response.status}: {user['email']} - Account deleted successfully")
        else:
            print(f"{response.status}: {user['email']}, response: {response.text}")

    rts.append(
        RT(
            method="delete",
            url=f'{BASE_URL_FUNCTIONS}/users/me',
            timeout=10,
            headers=tu.auth_json_headers(user["token"]),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

## Places

### Setup Places

In [ ]:
import test_data
importlib.reload(test_data)
places_data = test_data.get_places()

MIME_TYPE = 'image/jpeg'

user_pool = [
    tu.get_user_from_email("airidas.brikas@gmail.com", users),
    tu.get_user_from_email("rares@t.com", users),
    tu.get_user_from_email("anna@t.com", users),
    tu.get_user_from_email("elias@t.com", users),
]

places = []
for place_data in places_data:
    # Randomly assign a user to create each place
    user = random.choice(user_pool)

    place = {
        "action_caller": {"email": user['email']},
        "name": place_data["name"],
        "address": place_data["address"],
        "latitude": place_data["latitude"],
        "longitude": place_data["longitude"],
        "images": place_data.get("images", []),  # Get images from test_data if provided
        "test_id": place_data["test_id"],
        # "id": "..." will be populated after creation
    }
    places.append(place)

# Apply defaults
def apply_place_defaults():
    for place in places:
        if "images" not in place:
            place["images"] = []

apply_place_defaults()

print(f"Loaded {len(places)} places from test_data.py")
print(f"Places distributed among {len(user_pool)} users")

# Print statistics about images
places_with_images = [p for p in places if p["images"]]
print(f"{len(places_with_images)} places have images")

### Create places

In [ ]:
rts = []
ONLY_NON_EXISTING = True

for place in places:
    if "id" in place and ONLY_NON_EXISTING:
        print(f"Place {place['test_id']} already exists, skipping")
        continue

    user = tu.get_user_from_email(place['action_caller']['email'], users)

    form = FormData()
    form.add_field('name', place['name'])
    form.add_field('latitude', str(place['latitude']))
    form.add_field('longitude', str(place['longitude']))
    
    if 'address' in place and place['address']:
        form.add_field('address', place['address'])

    if 'images' in place:
        for img_path in place['images']:
            form.add_field(
                'images',
                open(img_path, 'rb'),
                filename=Path(img_path).name,
                content_type="image/jpeg"
            )

    def callback(response: bu.CustomResponse, user=user, place=place):
        if response.status_code == 200:
            r = response.json()
            if r != None:
                print(f"{response.status}: {place['name']} by {user['email']} -> {r['id']}")
                place['id'] = r['id']
        else:
            print(f"{response.status}: {place['name']} & {user['email']}, response: {response.text}")

    rts.append(
        RT(
            method="post",
            url=f'{BASE_URL_FUNCTIONS}/places',
            data=form,
            timeout=30,
            headers={
                "Authorization": f"Bearer {user['token']}",
            },
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False, max_concurrent=3)

### List places

In [ ]:
queries = [
    {
        "action_caller": tu.get_user_from_email("airidas.brikas@gmail.com", users),
        "lat": 36.373,
        "lng": 127.367,
        "radius_meters": 5000,
        "name": None,
        "sort_by": "distance",
        "sort_order": "asc",
    },
    {
        "action_caller": tu.get_user_from_email("rares@t.com", users),
        "lat": 36.3735,
        "lng": 127.3675,
        "radius_meters": 10000,
        "name": "pizza",
        "sort_by": "distance",
        "sort_order": "asc",
    },
]

rts = []

for query in queries:
    user = query['action_caller']

    def callback(response: bu.CustomResponse, user=user, query=query):
        if response.status_code == 200:
            r = response.json()
            if r != None:
                print(f"{response.status}: {user['email']} -> {len(r['results'])} places found")
                for place in r['results']:
                    print(f"  - {place['name']} ({place['distance_meters']:.0f}m)")
                    
                if r['current_page'] < r['total_pages']:
                    print(f"Fetching page {r['current_page'] + 1}/{r['total_pages']}...")
                    return response.get_request_template_copy_with_updated_params(
                        params_update={
                            "page": r['current_page'] + 1,
                        }
                    )
        else:
            print(f"{response.status}: {user['email']}, response: {response.text}")

    params = {
        "lat": query['lat'],
        "long": query['lng'],
        "radius_meters": query['radius_meters'],
        "sort_by": query['sort_by'],
        "sort_order": query['sort_order'],
    }
    if query['name']:
        params['name'] = query['name']

    rts.append(
        RT(
            method="get",
            url=f'{BASE_URL_FUNCTIONS}/places',
            params=params,
            timeout=10,
            headers=tu.auth_json_headers(user["token"]),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

### Get place details 

In [ ]:
queries = [
    {
        "action_caller": tu.get_user_from_email("airidas.brikas@gmail.com", users),
        "place_test_id": "subway-kaist",
        "lat": 36.3682,
        "lng": 127.3685,
    },
    {
        "action_caller": tu.get_user_from_email("rares@t.com", users),
        "place_test_id": "pizza-school",
        "lat": None,
        "lng": None,
    },
]

rts = []

for query in queries:
    user = query['action_caller']
    place = tu.get_item_from_test_id(query['place_test_id'], places)

    def callback(response: bu.CustomResponse, user=user, place=place):
        if response.status_code == 200:
            r = response.json()
            if r != None:
                print(f"{response.status}: {user['email']} -> {place['name']}")
                print(json.dumps(r, indent=2))
                place.update(r)
        else:
            print(f"{response.status}: {user['email']} & {place['name']}, response: {response.text}")

    params = {}
    if query['lat'] is not None and query['lng'] is not None:
        params['lat'] = query['lat']
        params['long'] = query['lng']

    rts.append(
        RT(
            method="get",
            url=f'{BASE_URL_FUNCTIONS}/places/{place["id"]}',
            params=params if params else None, # type: ignore
            timeout=10,
            headers=tu.auth_json_headers(user["token"]),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

### Update place

In [ ]:
place_updates = [
    {
        "action_caller": tu.get_user_from_email("airidas.brikas@gmail.com", users),
        "place_test_id": "subway-kaist",
        "form_data_str": {
            "name": "Subway KAIST - Updated",
            "address": "291 Daehak-rooooo, Yuseong-gu, Daejeon, South Korea",
        },
        "form_data_files": {
            "add_images": [
                # IMAGE_PLACE_3,
            ],
        },
        "remove_image_ids": [],
    },
    {
        "action_caller": tu.get_user_from_email("anna@t.com", users),
        "place_test_id": "pizza-school",
        "form_data_str": {
            "address": "123 New Address St, City, Country",
        },
        "form_data_files": {
            "add_images": [
                # IMAGE_PLACE_1,
                # IMAGE_PLACE_2,
            ],
        },
        "remove_image_ids": [],
    },
]

rts = []

for upd in place_updates:
    user = tu.get_user_from_email(upd['action_caller']['email'], users)
    place = tu.get_item_from_test_id(upd['place_test_id'], places)

    form = FormData()

    if 'form_data_str' in upd:
        for field, value in upd['form_data_str'].items():
            if isinstance(value, list):
                for v in value:
                    form.add_field(field, str(v))
            else:
                form.add_field(field, str(value))

    if 'form_data_files' in upd:
        for field, files in upd['form_data_files'].items():
            for file_path in files:
                form.add_field(
                    field,
                    open(file_path, "rb"),
                    filename=Path(file_path).name,
                    content_type="image/jpeg"
                )

    if upd.get('remove_image_ids'):
        remove_ids_str = ','.join(str(img_id) for img_id in upd['remove_image_ids'])
        form.add_field('remove_image_ids', remove_ids_str)

    def callback(response: bu.CustomResponse, user=user, place=place):
        if response.status_code == 200:
            r = response.json()
            print(f"{response.status}: {user['email']} -> {place['name']}")
            print(json.dumps(r, indent=2))
        else:
            if response.json() is not None:
                print(f"{response.status}: {user['email']} & {place['name']}")
                print(json.dumps(response.json(), indent=2))
            else:
                print(f"{response.status}: {user['email']} & {place['name']}, response: {response.text}")

    rts.append(
        RT(
            method="put",
            url=f"{BASE_URL_FUNCTIONS}/places/{place['id']}",
            data=form,
            headers={
                "Authorization": f"Bearer {user['token']}",
            },
            timeout=60,
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)